In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# QEC 101
## Lab 2 - Stabilizers, the Shor code, and the Steane code

$
\renewcommand{\ket}[1]{|{#1}\rangle}
\renewcommand{\bra}[1]{\langle{#1}|}
$
This lab introduces the stabilizer formalism, a powerful tool for working with more sophisticated quantum error correction (QEC) codes. After a brief introduction to the theory, the lab will walk through the Shor and Steane codes with interactive coding exercises. 

This lab was motivated by content from "[Quantum Error Correction: an Introductory Guide](https://arxiv.org/abs/1907.11157)" and "[Quantum Error Correction for Dummies](https://arxiv.org/abs/2304.08678)", both excellent resources we refer readers to for additional detail.  For a more technical introduction, see chapter 10 of "[Quantum Computation and Quantum Information](https://books.google.com/books?hl=en&lr=&id=-s4DEy7o-a0C&oi=fnd&pg=PR17&dq=quantum+computation+and+quantum+information&ots=NJ4KdqnzZt&sig=uKTETo5LLjWB9F_PV_zf0Sw3bvk#v=onepage&q=quantum%20computation%20and%20quantum%20information&f=false)" or the [PhD thesis](https://arxiv.org/abs/quant-ph/9705052) where the concept of stabilizer codes was introduced.

This is the second lab in the QEC series. If you are not familiar with the basics of classical or quantum error correction (EC), please complete the first lab in this series.

The list below outlines what you'll be doing in each section of this lab:

* **2.1** Define stabilizers and why they are important
* **2.2** Interactively Learn and Code the Steane Code in CUDA-Q.
* **2.3** Perform Steane Code Capacity Analysis with CUDA-QX
* **2.4** Interactively Learn and Code the Shor Code in CUDA-Q.



Lab 2 Learning Objectives:
* Understand what a stabilizer is, how it works, and why it is important
* Understand the approach of the Shor and Steane codes
* Understand logical operators
* Code the Shor and Steane codes in CUDA-Q

Execute the cells below to load all the necessary packages for this lab.

In [ ]:
## Instructions for Google Colab. You can ignore this cell if you have cuda-q set up and have 
# all the dependent files on your system
# Uncomment the lines below and execute the cell to install cuda-q

#!pip install cudaq
#!pip install cudaq_qec

#!wget -q https://github.com/nvidia/cuda-q-academic/archive/refs/heads/main.zip
#!unzip -q main.zip
#!mv cuda-q-academic-main/qec101/Images ./Images


In [ ]:
import cudaq
from cudaq import spin
from cudaq.qis import *
import numpy as np
import matplotlib.pyplot as plt
from typing import List

## 2.1 Stabilizers and Logical Operators


An important subclass of QEC codes, known as **stabilizer codes**, use special operations called **stabilizers** to clean up errors in encoded quantum information, and thus "stabilize" the state.


An operation $s$ acting on a state $\ket{\psi}$ is said to be a stabilizer of the state if the state is a +1 eigenstate of the operation $s \ket{\psi} = +1 \ket{\psi}$. The high-level intuiton here is that if small errors have accumulated in a logically encoded state, the action of applying this stabilizer is to project the state back to a perfectly error-free state, and we measure $+1$. Sometimes larger errors occur, and we do not measure $+1$, which informs us something has gone wrong.


In lab 1, the codespace was defined by the set of basis codewords, such as $\ket{000}$ and $\ket{111}$ for the 3-qubit quantum repetition code. In that lab the codewords were provided to you for each code, but in a stabilizer code, we can equivalently define the codespace by providing the stabilizers which stabilize each basis codeword.  In practice, this process of defining a code by the stabilizers is much more efficient and scalable as the codes grow larger.

The codespace $C$ can be defined as formed by all $\ket{\psi}$ such that $s_i\ket{\psi} = +1 \ket{\psi}$ for each $s_i\in S$, where these $s_i$ are stabilizers which form a group $S$ (note: in some texts this group $S$ is called the stabilizer, not the elements). That is, the codespace is the joint +1 eigenspace fixed by the stabilizers.  

Again in lab 1, we were given the codespace and error space for the 3-qubit quantum repetition code up front. However, let's think about working backwards from the computational basis for the 3-qubit Hilbert space. These can be sorted based on the eigenvalues returned when operated on by all elements of the stabilizer group $S = \{Z_1Z_2, Z_2Z_3\}$: 

| Basis state | Eigenvalue for $Z_1Z_2$ | Eigenvalue for $Z_2Z_3$ |
| ----------- | ----------- | ---------- | 
| $\ket{000}$ | 1 | 1 |
| $\ket{001}$ |  1 | -1 |
| $\ket{010}$ |  -1 | -1 |
| $\ket{100}$ |  -1 | 1 |
| $\ket{011}$ |  -1 | 1 |
| $\ket{101}$ |  -1 | -1 |
| $\ket{110}$ |  1 | -1 |
| $\ket{111}$ |  1 | 1 |




The basis states that have an eigenvalue of 1 for both $Z_1Z_2$ and $Z_2Z_3$ make up the codespace $\ket{000}$ and $\ket{111}$. There are other valid stabilizers in this code, such as $Z_1 Z_3$, but any stabilizer group can be boiled down into a minimal set which can be multiplied together to generate all of the others.

This is a really powerful approach, because it eliminates the need to derive and document the basis of the codespace in advance. Instead, one can simply define an appropriate set of stabilizers to establish the codespace.

Stabilizer codes are usually characterized as $[[n,k,d]]$ (double brackets for quantum codes) where $n$ is the number of physical qubits encoding $k$ logical qubits with distance $d$. It is always the case that these codes require $n-k$ stabilizers. The reason for this is that each stabilizer splits the original $2^n$ Hilbert space in two (the +1 and -1 eigenspace), with $2^1 = 2$ degrees of freedom remaining to define the logical qubit. 

The trick then becomes finding good sets of stabilizers that correspond to QEC codes with favorable properties.

### Stabilizer Properties 

Three key properties for $[[n,k,d]]$ stabilizers:

1. Here we consider only to Pauli product stabilizers, that is, $s_i$ needs to be a Pauli-group element. The n-qubit Pauli group $G_n$ is a special group constructed from the Pauli matrices:

  $$ I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}, \quad X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

  The group $G_n$ consists of $4*4^n$ elements and is built by forming a group that begins with all possible length $n$ Pauli words. For example $G_3$ has  terms like $IZZ$, $ZXZ$, $YYY$, etc. The group is then closed by including the possible scalar cofficients which arise from multiplying these terms $\{ 1, -1, i, -i\}$. So $G_1$ would be

  $$G_1 \equiv \{\pm I, \pm iI,\pm Z, \pm iZ,\pm X, \pm iX,\pm Y, \pm iY\}$$


2. Each $s_i$ must be able to operate on every logical state $\ket{\psi_L}$. Furthermore, this action should leave each $\ket{\psi_L}$ fixed (i.e., the eigenvalue of $\ket{\psi_L}$ should be +1 for each logical state).

3. Stabilizers need to be measurable in any order. This means each stabilizer needs to **commute** with every other stabilizer (that is, $s_is_j = s_js_i$, or equivalently $[s_i, s_j] \equiv s_is_j - s_js_i=0$).


### Logical Operators

In addition to stabilizers, each $[[n,k,d]]$ code has $2k$ **logical operators** ($\bar{X}_i$ and $\bar{Z}_i$) which perform $X$ and $Z$ operations on the logical states. For example:

$$ \bar{Z}_2\bar{X}_1\ket{11}_L = \bar{Z}_2\ket{01}_L = -\ket{01}_L $$

These logical operators must satisfy two properties. 

1. $\bar{X}_i$ and $\bar{Z}_i$ commute with all stabilizers.
2. $\bar{X}_i$ and $\bar{Z}_i$ must **anticommute** with one another if acting on the same logical qubit (i.e.,  $\{\bar{X}_i,\bar{Z}_j\}\equiv \bar{X}_i\bar{Z}_j + \bar{Z}_j\bar{X}_i= 2\delta_{ij}I$ for all $i,j$).


The next few sections will make use of stabilizers to solidify these concepts and enable coding of QEC codes far more interesting than the quantum repetition code. 

## 2.2 The Steane Code

The Steane code is a famous QEC code that is the quantum version of the [7,4,3] Hamming code introduced in the first QEC lab.  One immediate difference is that the Steane code encodes a single logical qubit making it a [[7,1,3]] code.

Remember, that the Hamming code adds additional parity bits that help "triangulate" where an error occurred. In the lab 1 exercises you constructed the generator matrix $G$ and used it to produce the logical codewords in the classical Hamming code. For example, $b=0110$ was encoded as


$$
c = bG=
 \begin{bmatrix} 0 & 1 & 1 & 0 \end{bmatrix}
\cdot
\begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{bmatrix}
=
\begin{bmatrix}
  0 & 1& 1 & 0 & 1& 1& 0
\end{bmatrix}
$$

Any logically encoded state, $c$, could then be multiplied by the parity check matrix ($H$) to determine if any syndromes were triggered or not. 


$$
Hc^T
=
\begin{bmatrix}
1 & 1 & 0 & 1 & 1 & 0 & 0 \\
1 & 0 & 1 & 1 & 0 & 1 & 0 \\
0 & 1 & 1 & 1 & 0 & 0 & 1
\end{bmatrix}
\cdot
\begin{bmatrix}
0 \\ 
1 \\ 
1 \\ 
0 \\ 
1 \\ 
1 \\ 
0
\end{bmatrix}
=
\begin{bmatrix}
0 \\ 
0 \\ 
0
\end{bmatrix}.
$$

What was not discussed in the Hamming code section was the fact that the parity check matrix ($H$) can be used to define the codespace. A valid logical codeword is any $c$ that satisfies the relationship $Hc^T=\begin{bmatrix}
0 \\ 
0 \\ 
0
\end{bmatrix}$.  As there are 7 data bits, that means there are $2^7=128$ possible encoded states between the codespace and error space.  It turns out, 16 of these fall within the codespace and 112 are in the error space. Of the 16 in the codespace, 8 have even parity (even number of 1's) while the other half has odd parity. 

| Even Bitstrings in Codespace | Odd Bitstrings in Codespace | 
| ----------- | ----------- |
| 0000000 | 1111111 |
| 0001111 | 1110000 |
| 0110110 | 1001001 |
| 0111001 | 1000110 |
| 1010101 | 0101010 |
| 1011010 | 0100101 |
| 1100011 | 0011100 |
| 1101100 | 0010011 |


This provides us a way to define the logical code words: $\ket{0}_L$ and $\ket{1}_L$.
The logical codewords, $\ket{0}_L$ and $\ket{1}_L$, for the Steane code are superpositions over the states corresponding to the classic even and odd codewords, respectively.



$$ \ket{0}_L = \frac{1}{\sqrt{8}}(\ket{0000000} +\ket{0001111} +\ket{0110110} + \ket{ 0111001} + \ket{1010101} + \ket{1011010}+ \ket{1100011} + \ket{1101100})  $$

$$ \ket{1}_L = \frac{1}{\sqrt{8}}(\ket{1111111} +\ket{1110000} +\ket{1001001} + \ket{1000110} + \ket{0101010} + \ket{0100101}+ \ket{0011100} + \ket{0010011})  $$


You might notice by inspection, that $\ket{0}_L = \bar{X}\ket{1}_L = X_1X_2X_3X_4X_5X_6X_7\ket{1}_L$.  That is to say, flipping all the bits swaps logical states.   Similarly, $\bar{Z} =  Z_1Z_2Z_3Z_4Z_5Z_6Z_7$ will flip the phase, transforming $\frac{1}{\sqrt{2}}(\ket{0}_L+\ket{1}_L)$ to $\frac{1}{\sqrt{2}}(\ket{0}_L-\ket{1}_L)$ 

The encoding circuit to produce the logical codewords is shown below, and is based off the constraints imposed by the parity check matrix. 

<img src="Images/steaneencoding.png" alt="Drawing" style="width: 500px;"/>

### Exercise 2.1 ###

In the cell below, build a CUDA-Q kernel to encode the logical 0 state.  Sample the circuit to prove that you indeed created the appropriate superposition.


In [ ]:
@cudaq.kernel
def steane_code():
    """Prepares a kernel for the Steane Code
    Returns
    -------
    cudaq.kernel
        Kernel for running the Steane code
    """   

    #Initialize Registers
    #TODO

    # Create a superposition over all possible combinations of parity check bits
    #TODO

    #Entangle states to enforce constraints of parity check matrix (circuit above)
    #TODO



results = cudaq.sample(steane_code, shots_count=100000)
print(results)   

The Steane code is a member of an important family of stabilizer codes known as **Calderbank-Shor-Steane (CSS)** codes. A CSS code is characterized by the property that Z and X errors can be detected and corrected independently.  A benefit of this, is that fewer ancilla qubits are required to produce the syndromes, and they can be reset and reused for each error. However, this procedure is also slower.

The stabilizers for $Z$-type errors are $S_Z = \{X_1X_2X_5X_4, X_1X_3X_4X_6, X_2X_3X_4X_7\},$ while the stabilizers for $X$-type errors are of similar form: $S_X = \{Z_1Z_2Z_5Z_4, Z_1Z_3Z_4Z_6, Z_2Z_3Z_4Z_7\}$.

Just as with the classical Hamming code, there is a nice way to visualize the syndrome results. The diagram below places each data qubit on each vertex. They are arranged such that each of the three sections or **plaquettes** corresponds to one of the stabilizers.  

The syndromes can be visually interpreted by putting a colored X on the syndromes that are flagged. Each coloring of this graph uniquely corresponds to an error on a specific qubit which is why the Steane code is often referred to as a **color code**.


<img src="Images/plaqettes.png" alt="Drawing" style="width: 700px;"/>

You are now ready to code the rest of the Steane code.  After encoding, introduce an $X$ error and $Z$ error on the qubits of your choice.  Try performing the $X$ and $Z$ syndrome measurements using the same three ancilla qubits and resetting them in between. Make your code such that you can measure the data qubits and confirm the state of the logical qubit.  

In [ ]:
import cudaq
@cudaq.kernel
def steane_code():
    """Prepares a kernel for the Steane Code
    Returns
    -------
    cudaq.kernel
        Kernel for running the Steane code
    """   

    #Initialize Registers
    #TODO

    # Create a superposition over all possible combinations of parity check bits
    #TODO

    #Entangle states to enforce constraints of parity check matrix (circuit above)
    #TODO

    #Add Errors (Optional)
    #TODO

    
    
    # Perform Stabilizer checks for Z errors
    #TODO


    # Perform Stabilizer checks for X errors
    #TODO


    # Correct X errors
    #TODO

    # Correct Z errors
    #TODO


results = cudaq.sample(steane_code, shots_count=1000)
print(results)  

#Post-process Results
#TODO

Now, test your code! Just measure in the $Z$ basis as the same procedure could be performed with the $X$ basis. 

1. Try adding single $X$ errors, guess which stabilizers should flag and confrm they do.
2. Add two errors. Confirm the code cannot correct the errors and a logical bitflip occurs.
3. It turns out that like the Shor code, there are alternate choices for $\bar{X}$.  Modify your counting code above and test if $X_0X_1X_4$ or $X_0X_4X_5$ are valid choices for $\bar{X}$.  

## 2.3 Steane Code Capacity Analysis with CUDA-Q QEC


[CUDA-QX](https://developer.nvidia.com/cuda-qx) is set of libraries that enable easy acceleration of quantum application development.  One of the libraries, [CUDA-Q QEC](https://nvidia.github.io/cudaqx/components/qec/introduction.html), is focused on error correction and can help expedite much of the work done above.  This final section will demonstrate how to run a code capacity memory experiment with the Steane code.

A memory experiment is a procedure to test how well a protocol can preserve quantum information. Such an experiment can help assess the quality of a QEC code but is often limited by assumptions that deviate from a realistic noise model. One such example is a code capacity experiment. A code capacity procedure determines the logical error rate of a QEC code under strict assumptions such as perfect gates or measurement.  Code capacity experiments can help put an upper bound on a procedure's threshold and is therefore a good starting place to compare new codes.

The process is outlined in the diagram below.  Assume the 0000000 bitstring is the baseline (no error).  Bitflips are then randomly introduced and produce errors in the data vector to produce results like 0100010.  If this were a real test on a physical quantum device, the data vector would not be known and a user could only proceed through the bottom path in the figure - performing syndrome extraction and then decoding the result to see if a logical flip occurred. In a code capacity experiment, the data vector with errors is known, so it can be used to directly compute if a logical state flip occurred or not.  Dividing the number of times the actual (top path) and predicted (bottom path) results agree by the total number of rounds provides an estimate of the logical error rate for the code being tested. 


<img src="Images/steanecodecapacity.png" alt="Drawing" style="width: 700px;"/>

CUDA-Q QEC allows researchers to streamline experiments like this with just a few lines of code.  Try running the cells below to compute the logical error rate of the Steane code under code capacity assumptions given probability of error $p$.


In [ ]:
import numpy as np                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
import cudaq_qec as qec 

Next, load the Steane code, which is already implemented in CUDA-Q QEC.

In [ ]:
steane = qec.get_code("steane")

The parity check matrices and observables can also be extracted from the Steane code.

In [ ]:
Hz = steane.get_parity_z()
Hx = steane.get_parity_x()
H = steane.get_parity()
observable  = steane.get_observables_z()

A decoder can then be specified which takes the parity check matrix as an input.

In [ ]:
decoder = qec.get_decoder("single_error_lut", Hz)

Then, `sample_code_capacity` can be called and provided with `p`, the probability of any bit flipping, and the number of shots for the analysis. 

In [ ]:
p = 0.01 # set a probability of a bit flip error occuring
nShots = 5000 # specify the number of shots
syndromes, data = qec.sample_code_capacity(Hz, nShots, p)

Notice how the Steane code is already defined within CUDA-Q QEC, along with a selection of decoders, and the `sample_code_capacity` API to automatically run the procedure.  Otherwise, you would need to code the entire process from scratch like you did in section 2.2 for each QEC code you want to test!

If the experiment is repeated many times with different $p$ values, a plot can be generated like the one shown below. The purple line is the $y=x$ and corresponds to the case that the logical error rate is identical to the physical error rate.  Anywhere the green line is below the purple line indicates that the Steane code was able to produce a logical error rate that is less than the physical error rate of the data qubits. When the green line is above the purple, the Steane code produced a worse logical error rate indicating that it would have been better to just use the data qubits and avoid the QEC procedure. The crossover point is an estimate for the code's threshold. Refining this estimate would require more sophisticated circuit level noise models that more accurately represent the performance of the Steane code under realistic conditions. 

<img src="Images/codecapacityplot.png" alt="Drawing" style="width: 700px;"/>

Though code capacity has much room to improve, it is a great example of the utility of CUDA-Q QEC and how simple procedures can be streamlined so users can focus on testing codes rather than coding up the details of each test.

## 2.4 The Shor Code

The first QEC code was proposed by Peter Shor in 1995, known as the [Shor code]((https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.R2493)).  The Shor code is a [[9,1,3]] code which uses 9 qubits to encode a single qubit, but can correct single $X$ or $Z$-type errors.


The motivation for the code, is that the 3-qubit repetition code can correct bit flip errors but not phase flip errors.  We can consider why this is by examining the encoded $\ket{+}_L$ state, which looks like the following:

$$ \ket{+}_L = \frac{1}{\sqrt{2}}(\ket{0}_{L~3\mathrm{bit}}+\ket{1}_{L~\mathrm{3bit}}) =  \frac{1}{\sqrt{2}}(\ket{000}+\ket{111}) $$

If a $Z_1$, $Z_2$, or a $Z_3$ error occurs, the $ \ket{+}_L$ state is transformed to $ \ket{-}_L$, another valid codeword.  This means there is no way to tell if a phase flip error occurred or not. One could produce the repetition code in the  $ \ket{+}_L$ and $ \ket{-}_L$ basis to correct $Z$ errors, but then the same problem would persist for $X$ errors.

The ingenuity behind the Shor code is to concatenate two 3-bit repetition codes into a 9-qubit code that can detect both types of errors. The encoding process begins with the 3-bit encoding of the $\ket{+}$ state.

$$ \ket{+}_{\mathrm{3 bit}} = \frac{1}{\sqrt{2}}(\ket{000}+\ket{111})$$

Then, $\ket{0}_L$ is encoded by taking a tensor product of three $\ket{+}_{\mathrm{3 bit}}$ states.


$$ \ket{0}_L = \ket{+}_{\mathrm{3 bit}} \otimes \ket{+}_{\mathrm{3 bit}} \otimes \ket{+}_{\mathrm{3 bit}} $$
$$ \ket{0}_L = \frac{1}{\sqrt{8}}(\ket{000} + \ket{111})(\ket{000} + \ket{111})(\ket{000} + \ket{111})$$

The same process is completed for the $\ket{1}_L$ state, this time using $\ket{-}_{\mathrm{3 bit}}$ as the starting point.
$$ \ket{1}_L = \frac{1}{\sqrt{8}}(\ket{000} - \ket{111})(\ket{000} - \ket{111})(\ket{000} - \ket{111})$$

This encoding of $\psi = \alpha \ket{0} + \beta \ket{1}$ can be implemented with the following quantum circuit:

<img src="Images/shorencode.png" alt="Drawing" style="width: 300px;"/>


The next consideration is to define the logical operators so that they behave as we expect, namely: $$\bar{X}\ket{0}_L = \ket{1}_L $$
$$\bar{X}\ket{1}_L = \ket{0}_L $$
and 
$$\bar{Z}\ket{0}_L = \ket{0}_L$$
$$\bar{Z}\ket{1}_L = -\ket{1}_L.$$
In other words, we need the following equations to hold:

$$\bar{X}\ket{0}_L = \bar{X}\frac{1}{\sqrt{8}}(\ket{000} + \ket{111})(\ket{000} + \ket{111})(\ket{000} + \ket{111}) = \frac{1}{\sqrt{8}}(\ket{000} - \ket{111})(\ket{000} - \ket{111})(\ket{000} - \ket{111}) = \ket{1}_L  $$


$$\bar{Z}\ket{1}_L = \bar{Z}\frac{1}{\sqrt{8}}(\ket{000} - \ket{111})(\ket{000} - \ket{111})(\ket{000} - \ket{111}) = \frac{1}{\sqrt{8}}(\ket{111} - \ket{000})(\ket{111} - \ket{000})(\ket{111} - \ket{000}) = -\ket{1}_L$$


Can you see what the logical operators need to be?  


For a logical bit flip to occur ($\bar{X}$) the phase of each block needs to change.  This is accomplished by performing a $Z $ operation on one of the qubits in each block, thus $\bar{X} = Z_1Z_4Z_7$ is a valid choice, though not the only choice as others like $\bar{X} = Z_2Z_5Z_8$ or even $\bar{X} = Z_1Z_2Z_3Z_4Z_5Z_6Z_7Z_8Z_9$ also work. Similarly, for $\bar{Z}$ to take $\ket{1}_L$ to $-\ket{1}_L$ (and $\ket{0}_L$ to itself) all of the bits need to flip, thus  $\bar{Z} = X_1X_2X_3X_4X_5X_6X_7X_8X_9$. The curious reader can confirm that the anticommutativity holds between these logical operators and that they commute with each stabilizer discussed below.


Now, consider what happens when $X$ and $Z$-type errors corrupt a state encoded with the Shor code. If a bitflip error occurs on qubit 8 of the $\ket{0}_L$ state.

$$ X_8\ket{0}_L = \frac{1}{\sqrt{8}}(\ket{000} + \ket{111})(\ket{000} + \ket{111})(\ket{010} + \ket{101})$$

The error only corrupts the third block of the code which houses the 8th qubit. So, this means stabilizers that perform parity checks on that block only  ($Z_7Z_8$ and $Z_8Z_9$) are sufficient to determine which position experienced an error. Extending this, all bit flip errors can be corrected with the following six stabilizers, two for each block.  Because each block is an independent repetition code, the Shor code can handle three bit flip errors,  as long as they occur in distinct blocks.

$$ S_{\mathrm{bit flips}} = \{Z_1Z_2, Z_2Z_3, Z_4Z_5, Z_5Z_6, Z_7Z_8, Z_8Z_9\} $$

Now consider the impact of a a phase flip error that acts on the 6th qubit, for example.

$$ Z_6\ket{0}_L = \frac{1}{\sqrt{8}}(\ket{000} + \ket{111})(\ket{000} - \ket{111})(\ket{000} + \ket{111})$$

The phase of the second block is changed which means the entire state can be rewritten as $\ket{+}_{\mathrm{3 bit}} \otimes \ket{-}_{\mathrm{3 bit}} \otimes \ket{+}_{\mathrm{3 bit}}$.  This "zoomed out" view makes it clear how the repetition code is leveraged again. This time a stabilizer is needed which can test the parity of block 1 with block 2 and block 2 with block 3. 

The stabilizer $X_1X_2X_3X_4X_5X_6$ can be used for this which will return 1 if the first two blocks have the same phase and -1 if they differ. Work this out by hand to convince yourself this works if it is not obvious why this is the case. Similarly, $X_4X_5X_6X_7X_8X_9$ can test the parity of the second two blocks completing the stabilizers necessary to detect phase flip errors.

$$ S_{\mathrm{phase flips}} = \{X_1X_2X_3X_4X_5X_6,X_4X_5X_6X_7X_8X_9   \} $$

All 8 stabilizers can correct any single-qubit $Z$ or $X$ error as summarized in the table below. Note that the Shor code is a redundant code, meaning that certain syndromes correspond to multiple errors.  At first this may seem problematic, but each error is fixed by the same correction, so knowing the specific source of the error is not always necessary.

| Error Type | Syndrome (Stabilizer Measurements) | 
| ----------- | ----------- |
| No Error | 0 0 0 0 0 0 0 0 |
| $X_1$ | 1 0 0 0 0 0 0 0 |
| $X_2$ | 1 1 0 0 0 0 0 0 |
| $X_3$ | 0 1 0 0 0 0 0 0 |
| $X_4$ | 0 0 1 0 0 0 0 0 |
| $X_5$ | 0 0 1 1 0 0 0 0 |
| $X_6$ | 0 0 0 1 0 0 0 0 |
| $X_7$ | 0 0 0 0 1 0 0 0 |
| $X_8$ | 0 0 0 0 1 1 0 0 |
| $X_9$ | 0 0 0 0 0 1 0 0 |
| $Z_1$ | 0 0 0 0 0 0 1 0 |
| $Z_2$ | 0 0 0 0 0 0 1 0 |
| $Z_3$ | 0 0 0 0 0 0 1 0 |
| $Z_4$ | 0 0 0 0 0 0 1 1 |
| $Z_5$ | 0 0 0 0 0 0 1 1 |
| $Z_6$ | 0 0 0 0 0 0 1 1 |
| $Z_7$ | 0 0 0 0 0 0 0 1 |
| $Z_8$ | 0 0 0 0 0 0 0 1 |
| $Z_9$ | 0 0 0 0 0 0 0 1 |




### Exercise 2.2

Now you have all of the backgound necessary to code the Shor code in CUDA-Q.  Fill in the sections below to build up a kernel that performs Shor code encoding and syndrome checks. The kernel should be constructed such that you can apply errors and select mesurement in the $Z$ or $X$ basis.

In [ ]:
import cudaq

@cudaq.kernel
def shor_code(error_qubit: list[int], error_location: list[int], measure: int):
    """Prepares a kernel for the Shor Code

    Parameters
    -----------
    error_qubit: list[int]
        a list where each element is an applied error designated as 1 =x or 2 =z
    error_location: list[int]
        each element corresponds to the index of the qubit which the error occurs on
    measure: int
        Option to measure in the z basis (0) or the x basis (1)

    Returns
    -------
    cudaq.kernel
        Kernel for running the Shor code
    """ 

    #Encode the data qubits with Shor encoding circuit.  Hint: It might be helpful to create separate registers for the data and ancilla qubits
    #TODO

    # Initial Psi (25/75) distribution in Z and X basis
    ry(1.04772,data_qubits[0])
    rz(1.521, data_qubits[0])

    # Apply optional single qubit errors 
    #TODO
    
    # Apply Hadamard gate to ancilla qubits 
    #TODO
    
    # Apply the Bit Flip syndromes 
    #TODO

    # Apply the phase flip syndromes 
    #TODO

    # Apply Hadamard gate to ancilla qubits 
    #TODO

    # Perform mid-circuit measurements to determine syndromes 
    #TODO
    
    
    # Apply the appropriate corrections based on the results from the syndrome measurements
    #TODO


    #Perform Hadamard on data qubits to rotate out of X basis (because of concatonated code)
    #TODO
    
    #Measure in X or Z basis depending on kernel input
    h(data_qubits) # put a Hadamard before the measurement to transform back into the Z basis
    # An X basis measurement can be obtained by applying a second Hadamard before a Z basis measurement
    #TODO

The final Hadamard is necessary because the code is concatenated and the second layer is in the $X$ basis.  Specifying the measurement basis allows you to confirm that the errors were or were not fixed.

You will also need to post process the results.  In the case of $Z$ basis measurement (where you see the impact of logical $X$ errors), you need to compute the parity of the logical $X$ operator $Z_1Z_2Z_3Z_4Z_5Z_6Z_7Z_8Z_9$, by measuring all the qubits in the $Z$ basis and computing the parity (Sum them and then mod 2) of the results.  

The same can be done for an $X$ basis measurement (where you see the impact of logical $Z$ errors). In this case you need to compute the parity of the logical $Z$ operator $X_1X_2X_3X_4X_5X_6X_7X_8X_9$ by measuring all of the qubits in the $X$ basis and computing thier parity.

Write a postprocessing function below which takes results, computes the parity of each measurment, prints the number of 1's and 0's, and prints the results.

In [ ]:
def post_process(results):
    """takes results from a CUDA-Q sample and prints the results and the number of 0's and 1's by computing the parity of the bitstrings.

    Parameters
    -----------
    results: cudaq.SampleResult
                A dictionary of the results from sampling the quantum state
    """


Now, run your code through the following tests and confirm it is working well. 

1. Prepare $\ket{\psi}$ in the $\ket{0}$ state.  Sample your kernel with no errors in the $Z$ and $X$ basis.  Do you see a 100/0 and 50/50 distribution for each respectively? What do you notice about the bitstrings when you measure in each basis?

2. Now, comment out the part of your code that fixes errors. Add a single $Z$ error and measure in the $Z$ and $X$ basis. Do you observe a bitflip in the $Z$ basis results?  Note, because the Shor code has an extra layer of Hadamard gates, a $Z$ error impacts the $Z$ observable which is not the usual case.

3. Now prepare $\ket{\psi}$ in the $\ket{+}$ state. Comment out the part of your code that fixes errors, add a single $X$ error, and measure in the $Z$ and $X$ basis. Do you observe a bitflip in the $X$ basis results now? 

4. Uncomment the part of your code that fixes the errors and run the same samples in 2 and 3. Did the correct syndrome flag and was the impact of the error ameliorated?

5. Prepare $\ket{\psi}$ in the $\ket{0}$ state again. With your full code, add multiple $Z$ errors and note that the stabilizer checks cannot properly fix them as the code is only distance 3.

In [ ]:
# TODO
# Run tests on your Shor Code